In [5]:
val deleteIntDataDF =  spark.read.option("multiline", "true").json("./Data/Json/meteorite_landing-delAvgRat.json")

deleteIntDataDF: org.apache.spark.sql.DataFrame = [data: array<array<string>>, meta: struct<view: struct<approvals: array<struct<reviewedAt:bigint,reviewedAutomatically:boolean,state:string,submissionDetails:struct<permissionType:string>,submissionId:bigint,submissionObject:string,submissionOutcome:string,submissionOutcomeApplication:struct<failureCount:bigint,status:string>,submittedAt:bigint,submitter:struct<displayName:string,id:string>,targetAudience:string,workflowId:bigint>>, assetType: string ... 37 more fields>>]


##### source: https://stackoverflow.com/questions/61863489/flatten-nested-json-in-scala-spark-dataframe/61863579#61863579
#####  The below dynamic code in scala is referred from the above source where it explodes the nested JSON into individual columns, since all of these are not arrays we cannot use the explode function and also it will make it more cumbersome to process individual columns. The below code dynamically splits arrays, and structs type into individual columns keeping its hierarchy intact, For eg if c is the nested child of b which is a child of an i.e. a.b.c, the code will split it as an a_b_c column. This will also prevent duplicating columns in case nested JSON has the same property name because its specific hierarchy will be attached to its name now.

In [6]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import scala.annotation.tailrec
import scala.util.Try

implicit class DFHelpers(df: DataFrame) {
    def columns = {
      val dfColumns = df.columns.map(_.toLowerCase)
      df.schema.fields.flatMap { data =>
        data match {
          case column if column.dataType.isInstanceOf[StructType] => {
            column.dataType.asInstanceOf[StructType].fields.map { field =>
              val columnName = column.name
              val fieldName = field.name
              col(s"${columnName}.${fieldName}").as(s"${columnName}_${fieldName}")
            }.toList
          }
          case column => List(col(s"${column.name}"))
        }
      }
    }

    def flatten: DataFrame = {
      val empty = df.schema.filter(_.dataType.isInstanceOf[StructType]).isEmpty
      empty match {
        case false =>
          df.select(columns: _*).flatten
        case _ => df
      }
    }
    def explodeColumns = {
      @tailrec
      def columns(cdf: DataFrame):DataFrame = cdf.schema.fields.filter(_.dataType.typeName == "array") match {
        case c if !c.isEmpty => columns(c.foldLeft(cdf)((dfa,field) => {
          dfa.withColumn(field.name,explode_outer(col(s"${field.name}"))).flatten
        }))
        case _ => cdf
      }
      columns(df.flatten)
    }
}



import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import scala.annotation.tailrec
import scala.util.Try
defined class DFHelpers


In [7]:
val exdeleteIntDataDF = deleteIntDataDF.explodeColumns

exdeleteIntDataDF: org.apache.spark.sql.DataFrame = [data: string, meta_view_approvals_reviewedAt: bigint ... 99 more fields]


In [8]:
spark.sql("ALTER TABLE icedsparkcatalog.mydb.meteorites SET TBLPROPERTIES ('write.spark.accept-any-schema'='true')")

res1: org.apache.spark.sql.DataFrame = []


In [9]:
val deleteIntAppendIcedTable = exdeleteIntDataDF.writeTo("icedsparkcatalog.mydb.meteorites").option("mergeSchema","true").append()

deleteIntAppendIcedTable: Unit = ()


In [10]:
val deleteIntIcedTable = spark.read.format("iceberg").load("icedsparkcatalog.mydb.meteorites")
deleteIntIcedTable.columns.length

deleteIntIcedTable: org.apache.spark.sql.DataFrame = [data: string, meta_view_approvals_reviewedAt: bigint ... 100 more fields]
res2: Int = 102


In [21]:
deleteIntIcedTable.select("meta_view_AverageRating").head(10)

res13: Array[org.apache.spark.sql.Row] = Array([null], [null], [null], [null], [null], [null], [null], [null], [null], [null])


In [22]:
deleteIntIcedTable.select("meta_view_AverageRating").tail(10)

res14: Array[org.apache.spark.sql.Row] = Array([0], [0], [0], [0], [0], [0], [0], [0], [0], [0])


In [18]:
deleteIntIcedTable.select("meta_view_AverageRating").filter("meta_view_AverageRating is null").count()

res10: Long = 542500
